In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

%matplotlib inline

# AdaBoost

## 9.1 Exponential loss vs misclasification loss

In boosting, we label (for convenience) the classes as $+1$ and $-1$, respectively. We furthermore let our classifier $\hat y(x)$ be on the form $\hat y(x) = \text{sign}(C(x))$, i.e., a thresholding at $0$ of some real-valued function $C(x)$. Assume that we have learned some function $\widehat{C}(x)$ from training data such that we can make predictions $\hat y(x) = \text{sign}(\widehat C(x))$. Fill in the missing columns in the table below:

| $\widehat{C}(x_\star)$ | $\hat{y}_\star$ | Exponential loss $\exp(-y_\star\widehat{C}(x_\star))$ | Misclassification loss $I(y_\star\neq\hat{y}_\star)$ | $y_\star$|
|:---:|:---:|:---:|:---:|:---:|
| 0.3  |      |      |      |  -1  |
|-0.2  |      |      |      |  -1  |
| 1.5  |      |      |      |   1  |
|-4.3  |      |      |      |   1  |

In what sense is the exponential loss more informative than the misclassification loss, and how can that information be used when training a classifier?

| $\widehat{C}(x_\star)$ | $\hat{y}_\star$ | Exponential loss $\exp(-y_\star\widehat{C}(x_\star))$ | Misclassification loss $I(y_\star\neq\hat{y}_\star)$ | $y_\star$|
|:---:|:---:|:---:|:---:|:---:|
| 0.3  |   1  |exp(0.3)=1.35|   1   |  -1  |
|-0.2  |  -1  |exp(-0.2)=0.82|   0   |  -1  |
| 1.5  |   1  |exp(-1.5)=0.22|   0   |   1  |
|-4.3  |  -1  |exp(4.3)=73.70|   1   |   1  |

Whereas the misclassification loss only gives the information `correct` or `misclassified`, the exponential loss contains the same information ($<1$: correct, $>1$ misclassfied) as well as information about the _margin_ $y_\star C(x_\star)$, which in a sense gives an idea of "how much" wrong or correct the classifier is.

## 9.2 AdaBoost for spam classification
Consider the very same setting data set for the data set `email.csv` as in problem 8.2, but now use AdaBoost instead. AdaBoost is available in `sklearn.ensemble` as the command `AdaBoostClassifier()`. What test error do you achieve?

In [ ]:
# url = 'data/email.csv'
url = 'https://github.com/uu-sml/course-sml-public/raw/master/data/email.csv'
email = pd.read_csv(url)

## 9.3 Exploring the AdaBoost algorithm
The script below illustrates the AdaBoost algorithm in the same way as was done in lecture 7. Familiarize yourself with the code and compare it to the pseudocode given in the course literature (Chapter 5 in the lecture notes). Explore what happens if you make changes in the input data and the number of trees (stumps), B, used!

## 9.4 Deriving the AdaBoost weights

The AdaBoost classifier can be written as 
$$ {\hat y_{\text{boost}}(x) = \text{sign}\left( C^B(x) \right)} $$
where the functions $C^1(x), \, \dots, C^B(x)$ are constructed sequentially as
$$ C^b(x) = C^{b-1}(x) + \alpha_b \hat y^b(x),$$
initialized with $C^0(x) \equiv 0$. The $b$th ensemble member $\hat y^b(x)$ is found by applying the chosen base classifier to a weighted version of the training data.
Once this has been found, we also need to compute the corresponding
"confidence" coefficient $\alpha_b$. This is done by minimizing the weighted exponential loss of the training data,
\begin{align}
\alpha_b = \arg\min_{\alpha}\left\{\sum_{i=1}^N w_i^b \exp(- \alpha y_i \hat y^b(x_i) ) \right\}
\end{align}
where $w_i^b = \exp(-y_i C^{b-1}(x_i))$.
		
Show that the optimal solution is given by
$$\alpha_b = \frac{1}{2}\log\left( \frac{1-\overline{\textrm{err}}_b}{\overline{\textrm{err}}_b} \right)$$
where 
$$\overline{\textrm{err}}_b = \sum_{i=1}^N  \frac{w_i^b}{\sum_{j=1}^N w_j^b} I(y_i \neq \hat G^b(x_i)).$$
		
_Hint 1: We use class labels $-1$ and $1$, i.e. $y_i \in \{-1, 1\}$ and $\hat y^b(x_i) \in \{-1, 1\}$. Using this fact, divide the sum in the objective function in the equation for $\alpha_b$ into one sum ranging over all correctly classified training data points and one sum ranging over all misclassified training data points._
		
_Hint 2: You are allowed to use the fact that the objective function in the equation for $\alpha_b$ has a single stationary point corresponding to the global minima._

We want to minimize the expression
$$F(\alpha) := \sum_{i=1}^N w_i^b \exp(- \alpha y_i \hat y^b(x_i) )$$
with respect to $\alpha$. Using the second hint of the problem formulation, we can do this by differentiating the expression, setting the derivative to zero, and solving for $\alpha$.

Before we do this, however, we make use of the first hint of the problem formulation to simplify the expression under study. Note first that using the class labels $-1$ and $1$ implies that
$$
y_i \hat G^b(x_i) =
\begin{cases}
1 & \text{if } \hat y^b(x_i) = y_i,\\
-1 & \text{if } \hat y^b(x_i) \neq y_i.
\end{cases}
$$
Thus, the expression for F(\alpha) can be written as
$$
F(\alpha) = e^{-\alpha} \underbrace{\sum_{i = 1}^N  w_i^b I( \hat y^b(x_i) = y_i)}_{=: W_c} + 
e^\alpha \underbrace{\sum_{i =1}^N  w_i^b I( \hat y^b(x_i) \neq y_i )}_{=: W_e},
$$
where we have used the indicator function to split the sum into two sums: the first ranging over all correctly classified data points, and the second ranging over all erroneously classified data points. 
Furthermore, for notational simplicity we define $W_c$ and $W_e$ for the sum of weights of correctly classified data points and erroneously classified data points, respectively.

Now, we compute the derivative and set to zero:
$$
\frac{dF}{d\alpha} = -e^{-\alpha} W_c + e^\alpha W_e = 0 \Longleftrightarrow
e^{\alpha} W_e = e^{-\alpha} W_c
\Longleftrightarrow
e^{2\alpha} = \frac{W_c}{W_e}		
\Longleftrightarrow
\alpha = \frac{1}{2} \log \left( \frac{W_c}{W_e} \right).
$$
It remains to write the solution on the format asked for in the problem formulation. With $W := \sum_{i=1}^N w_i^b$ denoting the sum of all weights, we note first that $W_c = W-W_e$ and thus,
\begin{align*}
\alpha = \frac{1}{2} \log \left( \frac{W - W_e}{W_e} \right)
= \frac{1}{2} \log \left( \frac{1 - W_e/W}{W_e/W} \right).
\end{align*}
Finally, noting that $\overline{\textrm{err}}_b = W_e/W$ completes the proof.

## 9.5  Gradient boosting
Explore gradient boosting by using `GradientBoostingClassifier()` on the spam data set `email.csv`